In [11]:
from typing import TypedDict
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph



In [12]:
# --- 1. Define shared state ---
class ChatState(TypedDict):
    input: str
    history: str
    response: str

In [13]:
memory = ConversationBufferMemory(return_messages=True)
# --- Node: Retrieve memory ----

def retrieve_history(state: ChatState) -> ChatState:
    past = memory.load_memory_variables({})       #loads fast memory
    state["history"] = past.get("history", "")    #stores it in history
    return state
# --- Node: Respond (dummy logic) ---
def generate_response(state: ChatState) -> ChatState:
    #response combining the new input and previous chat
    state["response"] = f"You said: {state['input']}. Previously, we talked about: {state['history']}"
    return state

# --- Node: Save to memory ---
def update_memory(state: ChatState) -> ChatState:
    #Saves the latest input-response to memory
    memory.save_context({"input": state["input"]}, {"output": state["response"]})
    return state


In [14]:
# --- Create graph ---
graph = StateGraph(ChatState)
graph.add_node("get_history", retrieve_history)
graph.add_node("chat", generate_response)
graph.add_node("save", update_memory)

graph.set_entry_point("get_history")
graph.add_edge("get_history", "chat")
graph.add_edge("chat", "save")
graph.set_finish_point("save")

app = graph.compile()

In [15]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [ ]:
# ---  Run multiple times to test memory ---
print("\nConversation 1:")
output1 = app.invoke({"input": "Hi there!", "history": "", "response": ""})
print(output1["response"])

print("\nConversation 2:")
output2 = app.invoke({"input": "What did I say before?", "history": "", "response": ""})
print(output2["response"])





Conversation 1:
You said: Hi there!. Previously, we talked about: [HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}), AIMessage(content='You said: Hi there!. Previously, we talked about: []', additional_kwargs={}, response_metadata={}), HumanMessage(content='What did I say before?', additional_kwargs={}, response_metadata={}), AIMessage(content="You said: What did I say before?. Previously, we talked about: [HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}), AIMessage(content='You said: Hi there!. Previously, we talked about: []', additional_kwargs={}, response_metadata={})]", additional_kwargs={}, response_metadata={})]

Conversation 2:
You said: What did I say before?. Previously, we talked about: [HumanMessage(content='Hi there!', additional_kwargs={}, response_metadata={}), AIMessage(content='You said: Hi there!. Previously, we talked about: []', additional_kwargs={}, response_metadata={}), HumanMessage(content='What did I sa